## Import libraries

In [1]:
import json
import requests
import pandas as pd
import datetime
import csv

## Set variables

In [23]:
BASE_URL = "https://api.figshare.com/v2"

In [53]:
categories = ["'Art'","'Digital Humanities'","'Disability Studies'","'Entertainment'","'Environmental Humanities'",
             "'Ethnic Studies'","'Gender Studies'","'History'","'Language'","'Law'","'Linguistics'","'Literature'",
             "'Media Studies'","'Museology'","'Performing Arts'","'Philosophy'","'Rhetoric'","'Religious Studies'"]

In [72]:
test = ["'Art'"] #86 and 150 Textile and Fashion Design

In [77]:
#Gather basic metadata for items (articles) that meet your search criteria
results = []
for i in categories:
    query = '{"search_for":":category: ' + i + '"}'
    y = json.loads(query) #Figshare API requires json paramaters
    for j in range(1,10):
        records = json.loads(requests.post(BASE_URL + '/articles/search?page_size=1000&page={}'.format(j), params=y).content)
        results.extend(records)
    
#See the number of articles
print(len(results),'articles retrieved')

22374 articles retrieved


In [80]:
#Save the json.
with open('humanities-records'+str(datetime.datetime.now().strftime("%Y-%m-%d"))+'.json', 'w') as f:
    json.dump(results, f)

In [112]:
#Create a dataframe from the JSON formatted data
dfall = pd.DataFrame(results)

In [115]:
dfunique = dfall.drop_duplicates(subset='id', keep="first")
print(len(dfall)-len(df),'records removed,',len(df),'unique records remain')

6001 records removed, 16373 unique records remain


In [134]:
#Remove records that have a handle not a doi
nan_value = float("NaN")
dfunique = dfunique.replace("", nan_value, inplace=True) #Replace blanks with nan
df = dfunique.dropna(subset = ["doi"])
print(len(dfunique)-len(df),'records removed,',len(df),'unique records remain')

325 records removed, 16048 unique records remain


c:\users\andre.desktop-dfg3gu2\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:4524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [137]:
#Save a file of all the metadata
save_file = df.to_csv('humanities-metadata'+str(datetime.datetime.now().strftime("%Y-%m-%d"))+'.csv')

In [2]:
#Open a file if needed
df = pd.read_csv('humanities-metadata2022-07-01.csv',encoding='utf8')

In [3]:
#Create a list of all the article ids
article_ids = df['id'].tolist()

In [11]:
article_ids2 = article_ids[12679:16047]
len(article_ids2)

3368

In [12]:
# Create a csv file, use an API to gather data, reopen the csv as a dataframe
   

#Create csv file
metadata=open('article-stats'+str(datetime.datetime.now().strftime("%Y-%m-%d"))+'.csv', 'w', newline='')
#Write header row to csv
csv.writer(metadata).writerow(['id','views','downloads','shares'])            

            
for l in article_ids2:
    s=requests.get('https://stats.figshare.com/total/views/article/'+ str(l),headers={"mailto":"andrew@figshare.com"})
    r=json.loads(s.text)
    t=requests.get('https://stats.figshare.com/total/downloads/article/'+ str(l),headers={"mailto":"andrew@figshare.com"})
    q=json.loads(t.text)
    u=requests.get('https://stats.figshare.com/total/shares/article/'+ str(l),headers={"mailto":"andrew@figshare.com"})
    v=json.loads(u.text)
    
    #write the values to the csv file. Dates in json files are seconds from jan 1 1970 so datetime.datetime.fromtimestamp converts
    csv.writer(metadata).writerow([
        l,
        r.get('totals'), #For any of these .get(), adding ",'N/A'" will fill the null cells with 'N/A'. However, metadata assessment counts non nulls
        q.get('totals'),
        v.get('totals')]) 
    
    
metadata.close() #Close the output file, release all locks

#Open up the same file as a dataframe. Encode cp1252 avoids a utf8 error.
dfstats = pd.read_csv('article-stats'+str(datetime.datetime.now().strftime("%Y-%m-%d"))+'.csv',encoding='utf8')

print('The resulting dataframe has',len(dfstats),'rows')

The resulting dataframe has 3368 rows


In [13]:
dfstats = pd.read_csv('article-stats-all-2022-07-02.csv',encoding='utf8')

In [15]:
df_full = df.merge(dfstats, how='inner', on='id')

In [17]:
#Save a file of all the metadata
save_file = df_full.to_csv('MASTER-humanities-metadata'+str(datetime.datetime.now().strftime("%Y-%m-%d"))+'.csv')